f.txt


In [ ]:
import os
import re
import glob 
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import numpy as np
from os.path import basename
import audiosegment
from multiprocessing import Pool
modulePath = '../../lib/ChristiansPythonLibrary/src' 
import sys
import numpy
sys.path.append(modulePath)
import generalUtility
import dspUtil
import matplotlibUtil
import librosa
import pickle
import string
from random import *
import cv2
min_char = 8
max_char = 20
allchar = string.ascii_letters + string.digits

def generate_random_string():
     return "".join(choice(allchar) for x in range(randint(min_char, max_char)))

default_dpi = 500
isAugmentData = False
    
#Augmentation factor
number_augmentated_per_image = 25
max_u_over_f = 3
u_over_f_values = numpy.arange(0.2, max_u_over_f, max_u_over_f / 40)
print(u_over_f_values)
scale_array = [ 1/ (np.abs((1-x)) + 0.1)    for x in u_over_f_values]
scale_array  = numpy.abs(scale_array)
print(scale_array)


EMOTION = {'ang': 0, 'hap' : 1, 'sad' : 2, 'neu' : 3, 'fru' : 4, 'exc': 5,
           'fea' : 6,'sur' : 7,'dis' : 8, 'oth' : 9, 'xxx':10}



In [ ]:
FOLDER_IMAGE_OUT = "processed-data-Emo"

def spectrogramToImage(freqs, times, amplitudes, dpi):
    fig, ax = plt.subplots(dpi = dpi)
    ax.pcolormesh(times, freqs, amplitudes)
    ax.axis('off')

    fig.canvas.draw ()
    data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    img = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)
    return img

def parallel_task(file_in):
    
    seg = audiosegment.from_file(file_in)
    freqs, times, amplitudes = seg.spectrogram(window_length_s=0.03, overlap=0.5)
    amplitudes = 10 * np.log10(amplitudes + 1e-9)

    img = spectrogramToImage(freqs, times, amplitudes, dpi=default_dpi)
    img = cv2.resize(img, (256, 256))
    
    raw_file_path = file_in.split("/")[-1].split(".")[0]
   
    
    cv2.imwrite(FOLDER_IMAGE_OUT + "/" + raw_file_path + ".png", img)
        
    if (isAugmentData == True):
        print("Augmenting data....")
        for x in scale_array:
        # plt.savefig(folder + "/" + generate_random_string() + file_out, transparent=True, dpi= int(default_dpi * x))
            img = spectrogramToImage(freqs, times, amplitudes, dpi= int(default_dpi * x))
            img = cv2.resize(img, (256, 256))
            cv2.imwrite(FOLDER_IMAGE_OUT + "/" + raw_file_path + ".png" ,img)

def createInputOutput():
    DATA_DIR = "../../auditary_emotion_recognition/EMO-DATA"
    FOLDER_AUDIO = "wav"
    files = []
    
    
    for f in os.listdir(DATA_DIR + "/" + FOLDER_AUDIO):
                if not f.startswith(".") and os.path.splitext(f)[1] == ".wav":
                    files.append(DATA_DIR + "/" + FOLDER_AUDIO + "/" + f)
    
    # Multi processing
    with Pool(processes=8) as pool:
        pool.map(parallel_task, files)
   
    print("Finished create histogram into files")
    
    
createInputOutput()
    
    